# Benchmarking

Benchmarking or profiling are used to measure the speed with which you code can run and identify bottlenecks.

## timeit

Usesd to time small code snippets.

In [1]:
# via commandline
# run timeit and setup then gives average of executing loop 5 times

!python -m timeit -s '[ord(x) for x in "abcdfghi"]'

50000000 loops, best of 5: 6 nsec per loop


In [2]:
# another example using the commandline

!python -m timeit -s '[chr(int(x)) for x in "123456789"]'

50000000 loops, best of 5: 6.21 nsec per loop


In [3]:
!ls

10_typing_module.ipynb           3_context_managers.ipynb
11_python_builtins.ipynb         4_functools_module.ipynb
12_unicode.ipynb                 5_imports.ipynb
13_benchmark_demos               6_import_lib_demos
13_benchmarking.ipynb            6_importlib_module.ipynb
1_arg_demos                      7_iterators_and_generators.ipynb
1_argparse.ipynb                 8_itertools.ipynb
2_collections_demos              9_regular_expressions.ipynb
2_collections_module.ipynb       README.md
3_context_manager_demos          test.db


In [4]:
# importing a function and benchmarking it
%cd 13_benchmark_demos 
!python -m timeit "import simple_func; simple_func.my_function()"
%cd ..

/Users/miesner.jacob/python-for-programmers-educative/Module 4 - Advanced Concepts in Python/13_benchmark_demos
1000000 loops, best of 5: 363 nsec per loop
/Users/miesner.jacob/python-for-programmers-educative/Module 4 - Advanced Concepts in Python


## importing timeit for Testing

In [9]:
if __name__ == "__main__":
    import timeit
    setup = "from __main__ import my_function"
    print(timeit.timeit("my_function()", setup=setup))

0.3262987550000105


In [10]:
## Use a decorator
## Writing our own timing function
import random
import time

def timerfunc(func):
    """
    A timer decorator
    """
    def function_timer(*args, **kwargs):
        """
        A nested function for timing other functions
        """
        start = time.time()
        value = func(*args, **kwargs)
        end = time.time()
        runtime = end - start
        msg = "The runtime for {func} took {time} seconds to complete"
        print(msg.format(func=func.__name__,
                         time=runtime))
        return value
    return function_timer


@timerfunc
def long_runner():
    for x in range(5):
        sleep_time = random.choice(range(1,5))
        time.sleep(sleep_time)

if __name__ == '__main__':
    long_runner()

The runtime for long_runner took 12.0014169216156 seconds to complete


## Create a Timing Context Manager

In [11]:
## Using a context manager to time code

import random
import time

class MyTimer():

    def __init__(self):
        self.start = time.time()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        end = time.time()
        runtime = end - self.start
        msg = 'The function took {time} seconds to complete'
        print(msg.format(time=runtime))


def long_runner():
    for x in range(5):
        sleep_time = random.choice(range(1,5))
        time.sleep(sleep_time)


if __name__ == '__main__':
    with MyTimer():
        long_runner()

The function took 13.014636993408203 seconds to complete


## cProfile

Python comes with the profile and cProfile modules built-in. Cprofile is recommended because it is much faster.

In [15]:
import cProfile
cProfile.run("[x for x in range(100000000)]")

         4 function calls in 6.749 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    4.180    4.180    4.180    4.180 <string>:1(<listcomp>)
        1    2.567    2.567    6.748    6.748 <string>:1(<module>)
        1    0.001    0.001    6.749    6.749 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




* **ncalls** is the number of calls made.
* **tottime** is a total of the time spent in the given function.
* **percall** refers to the quotient of tottime divided by ncalls
* **cumtime** is the cumulative time spent in this and all subfunctions. It’s even accurate for recursive functions!
* The second **percall** column is the quotient of cumtime divided by primitive calls
* **filename:lineno(function)** provides the respective data of each function

In [21]:
# Using cProfile via commandline!

%cd 13_benchmark_demos 
!python -m cProfile test.py
%cd ..

/Users/miesner.jacob/python-for-programmers-educative/Module 4 - Advanced Concepts in Python/13_benchmark_demos
         4 function calls in 0.534 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.534    0.534    0.534    0.534 test.py:1(<listcomp>)
        1    0.000    0.000    0.534    0.534 test.py:1(<module>)
        1    0.000    0.000    0.534    0.534 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


/Users/miesner.jacob/python-for-programmers-educative/Module 4 - Advanced Concepts in Python


## line_profiler

line_profiler is a python module to seehow long it take for each line in your file to run, it also has a script called karnprof for profiling Python code.

In [7]:
!pip install line_profiler

In [3]:
%cd 13_benchmark_demos

/Users/miesner.jacob/python-for-programmers-educative/Module 4 - Advanced Concepts in Python/13_benchmark_demos


In [4]:
!ls

__pycache__        silly_functions.py simple_func.py     test.py


In [10]:
# Using kernprof in verbose mode to profile our python file line by line

!kernprof -l -v silly_functions.py 

I'm a fast function!
I'm a slow function
Wrote profile results to silly_functions.py.lprof
Timer unit: 1e-06 s

Total time: 2.7e-05 s
File: silly_functions.py
Function: fast_function at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           @profile
     5                                           def fast_function():
     6         1         27.0     27.0    100.0      print("I'm a fast function!")

Total time: 2.00447 s
File: silly_functions.py
Function: slow_function at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           @profile
     9                                           def slow_function():
    10         1    2004413.0 2004413.0    100.0      time.sleep(2)
    11         1         55.0     55.0      0.0      print("I'm a slow function")



There are six columns of output, here is what they all mean:

1. Line # - Line number of code profiled
2. Hits - Number of times line was executed
3. Time - The total amount of time the line took to execute
4. Per Hit - The average amount of time the line took to execute
5. % Time - The relative percent of total time the line took to execute
6. Line Contents -THe actual source code

(IPython has a magic command (%lprun) that allows you to specify functions to profile and even which statement to execute.)

## Memory Profiler

A 3rd party profiling package for monitoring memory consumption line by line.

In [21]:
!pip install memory_profiler

In [24]:
# Running memory profiling

!python3 -m memory_profiler memo_prof.py 

Filename: memo_prof.py

Line #    Mem usage    Increment  Occurences   Line Contents
     3   39.176 MiB   39.176 MiB           1   @profile
     4                                         def mem_func():
     5   39.191 MiB    0.016 MiB           1       lots_of_numbers = list(range(1500))
     6  113.699 MiB   74.508 MiB           1       x = ['letters'] * (5 ** 10)
     7  113.699 MiB    0.000 MiB           1       del lots_of_numbers
     8  113.699 MiB    0.000 MiB           1       return None




In [25]:
# Full Memory usage reports over time instead of line by line
!mprof run memo_prof.py 

mprof: Sampling memory every 0.1s
running new process
running as a Python program...


In [28]:
# Plotting memory usage
# This wont print out the plot in an ipython notebook but will on commandline
!mprof plot

Using last profile data.
Figure(1260x540)


## proflehooks

Another 3rd party package with a collection of decorators for profling

In [29]:
!pip install profilehooks

In [31]:
# Example including just the @profile decorator
!python3 profhooks.py


*** PROFILER RESULTS ***
mem_func (profhooks.py:5)
function called 1 times

         2 function calls in 0.030 seconds

   Ordered by: cumulative time, internal time, call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.030    0.030    0.030    0.030 profhooks.py:5(mem_func)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        0    0.000             0.000          profile:0(profiler)




In [32]:
# Using the timecall decorator that goves the course run time of the function its decorating
!python3 profhooks2.py


  mem_func (profhooks2.py:4):
    0.057 seconds

